In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')


#clear out download directories prior in the case or re-runs
# Example usage

sbac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\sbac\{formatted_month_day_year}'
elpac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\elpac\{formatted_month_day_year}'
sbac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\sbac_{formatted_month_day_year}'
elpac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\elpac_{formatted_month_day_year}'


for i in [sbac_local_dir, elpac_local_dir, sbac_pdrive_dir, elpac_pdrive_dir]:
    empty_directory(i)
    create_directory(i)

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


def selenium_process(SY):

    logIn(username, password, driver)
    launch_to_homescreen(driver)

    # ---------------------------------------SBAC & ELPAC Files Request and Download-------
    # Call the function, school report names variable is called for just school name. MUst occur in this order for Selenium
    #Equivalent of Student Score Data File
    SBAC_output = SBAC_package_func(driver, SY, 'Tested', formatted_month_day_year)
    ELPAC_output = ELPAC_package_func(driver, SY, 'Tested', formatted_month_day_year)

    return(SBAC_output, ELPAC_output)

SBAC_output, ELPAC_output = selenium_process('2024')

# --------------------------------Unzip the XLSX Files and Move them to the P-Drive, Additional Unit Test---------------Path - 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'

SBAC_output = unzip_move_and_unit(SBAC_output, 'sbac', formatted_month_day_year)
ELPAC_output = unzip_move_and_unit(ELPAC_output, 'elpac', formatted_month_day_year)

# # ---------------------------------POST SELENIUM PROCESS, STACKING & SENDING FILES----------------------------------

sbac_stack = stack_files(sbac_pdrive_dir, 'CAASPP') #This is where files are raw and stacked before transformation
elpac_stack = stack_files(elpac_pdrive_dir, 'ELPAC') #Green Dot Schools are pulled out of master file

# # -----------------------------Where the normalization of the dataframes occur, column changing & mapping------------------
# elpac = get_elpac_import(elpac_stack, 'ELPAC')
sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall 
cast = get_cast_import(sbac_stack, 'CAST')

# #For Helens Ellevation Pickup, send to same dir as individual files the stack in stacked_files dir
# send_stacked_csv(elpac, 'ELPAC', formatted_month_day_year) 
send_stacked_csv(sbac, 'SBAC', formatted_month_day_year)
send_stacked_csv(cast, 'CAST', formatted_month_day_year)

# # -----------------------------------------------Send over new records------------------------

def send_to_sql(frame, file_name):
    dtypes, table_cols = SQL_query.get_dtypes(frame, 'DataTeamSandbox', f'{file_name}_Scores')

    # Reference the DataTeamSandbox master tables before they are fully replaced with today's update in order to find the incoming records
    #These are populated within the dictionary before the master table is updated. Therefore they are good. 
    new_records = {
        'CAST': grab_new_records(cast, 'CAST'),
        # 'ELPAC': grab_new_records(elpac, 'ELPAC'),
        'SBAC': grab_new_records(sbac, 'SBAC')
    }
    
    #Update the master table with a full replace, after assessing todays incoming records by each table
    try:
        frame.to_sql(f'{file_name}_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'replace', index = False, dtype=dtypes)
        logging.info(f"Sent data - {len(frame)} records to master table {file_name}_Scores")
    except Exception as e:
        logging.info(f'Unable to send data to {file_name}_Scores due to \n {e}')

    #Update the table with append of only new records, and timestamp it within new_records func
    try:
        new_records[file_name].to_sql(f'{file_name}_New_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'append', index = False, dtype=dtypes)
        logging.info(f"Sent data to {file_name}_New_Scores, by appending {len(new_records[file_name])} new records")
    except Exception as e:
        logging.info(f'Unable to send data to {file_name}_New_Scores due to \n {e}')


#  # OBTAINING NEW RECORDS PROCESS
# # The master tables get a full replace with todays data files, however this does not occur until todays data files
# # is compared to the master tables. 

# #Whatever is strictly coming in on the merge from the new frame from these 4 columns will be sent to new scores table
# # ['SSID', 'TestType', 'TestName', 'ScaleScore']

# #After new scores table is appended with new records with last_update timestamp, the master table gets a full replace of
# #todays data files. 


# send_to_sql(elpac, 'ELPAC')
send_to_sql(sbac, 'SBAC')
send_to_sql(cast, 'CAST')

In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


e = get_elpac_import(elpac_stack, 'ELPAC')

es = get_SS_frame(e)
ep = get_PL_frame(e)
em = pd.merge(ep, es, left_on=['SSID', 'TestName'], right_on = ['SSID', 'TestName'], suffixes= ['', '_SS'], how='left')

#Original Frame
e = e[['SSID', 'OverallScaleScore', 'OralLanguageScaleScore','WrittenLanguageScaleScore']]
e.loc[e['SSID'] == 1367208652]

#Melted Frame
em = em[['SSID', 'TestName', 'ScaleScore']]
em.loc[em['SSID'] == 1367208652]

In [ ]:
em = pd.merge(ep, es, left_on=['SSID', 'TestName'], right_on = ['SSID', 'TestName'], suffixes= ['', '_SS'], how='left')

# Testing PL Score ScaleScore Creation for ELPAC

In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.post_download_change import *



sbac_stack = stack_files(sbac_pdrive_dir, 'CAASPP') #This is where files are raw and stacked before transformation
elpac_stack = stack_files(elpac_pdrive_dir, 'ELPAC') #Green Dot Schools are pulled out of master file


# scale_score = get_SS_frame(elpac_stack)
# pl_score = get_PL_frame(elpac_stack)

# print(len(scale_score))
# print(len(pl_score))



# elpac = get_elpac_import(elpac_stack, 'ELPAC')
# sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall 
# cast = get_cast_import(sbac_stack, 'CAST')
#Look at AchievementLevels for SBAC

In [ ]:
print(len(scale_score['SSID'].unique()))
print(len(pl_score['SSID'].unique()))
#Multi key dictionary based on all the same SSIDs. 


# Set multi-index and convert to dictionary
scale_score_dict = scale_score.set_index(['SSID', 'TestName'])['ScaleScore'].to_dict()

# Function to map ScaleScore using the dictionary
def map_scale_score(row, score_dict):
    return score_dict.get((row['SSID'], row['TestName']))

# Apply the function to create a new ScaleScore column in pl_score DataFrame
pl_score['ScaleScore'] = pl_score.apply(map_scale_score, score_dict=scale_score_dict, axis=1)

pl_score[['SSID', 'TestName', 'ScaleScore', 'PLScore']].isnull().value_counts()

#Still gets the same amount of nulls

In [ ]:
pl_score['ScaleScore'].isnull().value_counts()

#Still only 4503 ScaleScores. WHy is that. 
#Figure out a way to map PL Scores based on something else. 

#Try on a smaller scale. Try melting down the same frame twice on a smaller scale. 

In [ ]:
e = elpac_stack[['SSID', 'OverallScaleScore', 'OralLanguageScaleScore','WrittenLanguageScaleScore', 'OverallPL','OralLanguagePL', 'WrittenLanguagePL', 'ListeningPL', 'SpeakingPL', 'ReadingPL', 'WritingPL' ]].head(10)
es = get_SS_frame(e)

ep = get_PL_frame(es)
ep.loc[ep['SSID'] == 1367208652]
#This causes repeats of ScaleScores. 
#SSIDs repeat 21 times

# Running with Todays File Downloads

In [ ]:
#Debug get_elpac_import
%load_ext autoreload
%autoreload 2
#Debug get_elpac_import
from config import username, password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
pd.set_option('display.max_columns', None)



# ---------------------------------POST SELENIUM PROCESS, STACKING & SENDING FILES----------------------------------

directory_path = r'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'

directory_path_sbac = os.path.join(directory_path, f'sbac_{formatted_month_day_year}')
sbac_stack = stack_files(directory_path_sbac)

directory_path_elpac = os.path.join(directory_path, f'elpac_{formatted_month_day_year}')
elpac_stack = stack_files(directory_path_elpac)

elpac_stack = filter_on_full_cds_code(elpac_stack, 'CALPADSSchoolCode')
sbac_stack = filter_on_full_cds_code(sbac_stack, 'CALPADSSchoolCode')
# elpac_stack = pd.read_csv('file_downloads\elpac_stack.csv') #For testing purposes to start from this point
# sbac_stack = pd.read_csv('file_downloads\sbac_stack.csv')

# -----------------------------Where the normalization of the dataframes occur, column changing & mapping------------------
elpac = get_elpac_import(elpac_stack, 'ELPAC')
sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall 
cast = get_cast_import(sbac_stack, 'CAST')

#For Helens Ellevation Pickup, send to same dir as individual files the stack in stacked_files dir
send_stacked_csv(elpac, 'ELPAC', directory_path, formatted_month_day_year) 
send_stacked_csv(sbac, 'SBAC', directory_path, formatted_month_day_year)
send_stacked_csv(cast, 'CAST', directory_path, formatted_month_day_year)

# -----------------------------------------------Send over new records------------------------

def send_to_sql(frame, file_name):
    dtypes, table_cols = SQL_query.get_dtypes(frame, 'DataTeamSandbox', f'{file_name}_Scores')

    # Reference the DataTeamSandbox master tables before they are fully replaced with today's update in order to find the incoming records
    #These are populated within the dictionary before the master table is updated. Therefore they are good. 
    new_records = {
        'CAST': grab_new_records(cast, 'CAST'),
        'ELPAC': grab_new_records(elpac, 'ELPAC'),
        'SBAC': grab_new_records(sbac, 'SBAC')
    }
    
    #Update the master table with a full replace, after assessing todays incoming records by each table
    try:
        frame.to_sql(f'{file_name}_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'replace', index = False, dtype=dtypes)
        logging.info(f"Sent data - {len(frame)} records to master table {file_name}_Scores")
    except Exception as e:
        logging.info(f'Unable to send data to {file_name}_Scores due to \n {e}')

    #Update the table with append of only new records, and timestamp it within new_records func
    try:
        new_records[file_name].to_sql(f'{file_name}_New_Scores', schema='dbo', con = SQL_query.engine, if_exists = 'append', index = False, dtype=dtypes)
        logging.info(f"Sent data to {file_name}_New_Scores, by appending {len(new_records[file_name])} new records")
    except Exception as e:
        logging.info(f'Unable to send data to {file_name}_New_Scores due to \n {e}')


 # OBTAINING NEW RECORDS PROCESS
# The master tables get a full replace with todays data files, however this does not occur until todays data files
# is compared to the master tables. 

#Whatever is strictly coming in on the merge from the new frame from these 4 columns will be sent to new scores table
# ['SSID', 'TestType', 'TestName', 'ScaleScore']

#After new scores table is appended with new records with last_update timestamp, the master table gets a full replace of
#todays data files. 


send_to_sql(elpac, 'ELPAC')
send_to_sql(sbac, 'SBAC')
send_to_sql(cast, 'CAST')

# Attempting to get files without specifying dates. See if the files come across. 